# Paris births model

Let us construct the Paris births model using Stan.

We'll use `pystan`, the Python interface to Stan, and also a package called `jupyterstan`, written by former Faculty data scientist Jan Freyberg, which gives us Stan syntax highlighting inside jupyter. See below for some details on how to use `pystan` with a seperate `.stan` file.

In [ ]:
!pip install arviz pystan jupyterstan

In [ ]:
import pystan

%load_ext jupyterstan

## Data

In [ ]:
FEMALE_BIRTHS = 241945
MALE_BIRTHS = 251527

## Specifying the Stan model

We let $n$ denote the total number of births, $y$ the number of female births, and $p$ the underlying probability that any given baby is born female (which are assumed independent, identically distributed events). The model we are going to fit can be specified as:

$$
    \theta \sim \mathrm{Unif}(0,1) \\
    y | \theta \sim \mathrm{Binomial}(n, \theta)
$$

i.e. we assume that all possible values of $\theta$ are equally likely, and that $y$ given $\theta$ is distributed binomiall with $n$ trials and probability $\theta$.

Let's specify this model in Stan. Thanks to `jupyterstan` we can use the `%%stan` magic to specify a Stan code cell which gives us syntax higlighting. The compiled model will be saved to the `female_birth_model` variable.

In [ ]:
%%stan female_birth_model

data {
    int n;  // total births
    int y;  // female births
}

parameters {
    real<lower=0, upper=1> theta;
}

model {
    // 
    y ~ binomial(n, theta);
}

Hit enter on the cell above. It will take a bit of time to compile. For simple models like this the compilation time is actually much longer than the sampling time. When it's done, we should have a new `StanModel` object assigned to `female_birth_model`.

In [ ]:
female_birth_model

We can now pass our data to the model and ask it to produce samples for us using the `sampling` method of the model object.

In [ ]:
# data is passed to Stan as a dict
data = {
    "n": FEMALE_BIRTHS + MALE_BIRTHS,
    "y": FEMALE_BIRTHS,
}

fit = female_birth_model.sampling(data=data, chains=4, iter=4000, n_jobs=1)
print(fit)

As you can see above, printing the fit object gives us basic summary stats of the sampling, such as the sample mean, mean standard error (estimated sampling error), the sample standard deviation, and various sample quantiles that let us easily see what the central 50% and 95% intervals are for each parameter (in our case, we only have one parameter p, lp__ is the log posterior which will get generated by every Stan model). In addition to summary statistic, fit displays two diagnostic metrics:

 * n_eff - Stan's sampling algorithm draws correlated samples, which have less statistical power than independent samples. This metric estimates how many independent draws from the posterior distribution would have the same power as the correlated samples Stan has drawn.
 * Rhat - This is a convergence metric for the sampling algorithm. If all of your chains have converged it will be 1 or very close to it. If you have Rhat greater than 1.2 you should probably investigate, if it’s greater than 2 your samples are probably no good for inference. Note that Rhat equal to 1 doesn’t actually guarantee convergence, it is a necessary condition but not sufficient.

To use the sampled values themselves, run the `extract` method

In [ ]:
params = fit.extract()

params["theta"]

## Visualising the fit

Use `arviz` to easily visualise your fitted parameters

In [ ]:
import arviz as az

az_data = az.from_pystan(posterior=fit)

Once you've created the `az_data` object, you can plot the sampled density of any of the parameters. The plot will be truncated at the credible interval.

In [ ]:
az.plot_density(az_data, var_names=["theta"], figsize=(12, 8));

There are a number of other plots available, including a forest, which lets you easily compare chains

In [ ]:
az.plot_forest(az_data, figsize=(12, 5));

Or a trace plot, which again gives us a sense of whether the sampler converged.

In [ ]:
az.plot_trace(az_data);

## Exercises

1. The average height of males in the UK is 175cm, with a standard deviation of 10cm. Use `numpy.random.normal` to generate 10 random normal samples with mean 175 and standard deviation 10.

In [ ]:
import numpy as np

samples = np.random.normal(loc=175, scale=10, size=10)

2. Suppose that we didn't know the true mean height, which we call `mu`, but we know somehow that the standard deviation is 10cm. We want to try and infer the mean from data. We decide a normal prior with mean 150 and standard deviation 50 (so that we are 99% sure that the mean is going to lie between 0cm and 3m) will be reasonably informative but not introduce too much bias. Hence our model is
$$
    \mu \sim \mathcal{N}(150, 50^2) \\
    h \sim \mathcal{N}(\mu, 10^2)
$$
Code this model up in Stan, and then draw samples from the posterior distribution for `mu`. Plot the posterior density using `plot_density` from `arviz`.

In [ ]:
%%stan normal_model
data {
  int n;
  vector[n] h;
}
parameters {
  real mu;
}
model {
  mu ~ normal(150, 50);
  h ~ normal(mu, 10);
}

In [ ]:
data = {
    "n": len(samples),
    "h": samples,
}

fit = normal_model.sampling(data=data, chains=4, iter=2000, n_jobs=1)

3. Estimate the following:

    a. The posterior mean of `mu`.
    
    b. The probability that `mu` is greater than 175cm.
    
    c. The 95% credible interval for `mu`.

In [ ]:
mu = fit.extract()["mu"]

post_mean = mu.mean()
print(f"The posterior mean is approx {post_mean}")

prob_geq_175 = (mu >= 175).sum() / mu.shape
print(f"The probability mu >= 175 is approx {prob_geq_175}")

int95 = np.percentile(mu, [2.5, 97.5])
print(f"95% credible interval is approx {int95}")

4. Repeat the above with a random sample of 10000 heights from $\mathcal{N}(175, 10^2)$

In [ ]:
samples_10k = np.random.normal(175, 10, 10000)

data_10k = {
    "n": len(samples),
    "h": samples,
}

fit_10k = normal_model.sampling(data=data, chains=4, iter=2000, n_jobs=1)

mu_10k = fit_10k.extract()["mu"]

post_mean_10k = mu_10k.mean()
print(f"The posterior mean is approx {post_mean}")

prob_geq_175_10k = (mu_10k >= 175).sum() / mu.shape
print(f"The probability mu >= 175 is approx {prob_geq_175_10k}")

int95_10k = np.percentile(mu_10k, [2.5, 97.5])
print(f"95% credible interval is approx {int95_10k}")

## Credit

This notebook makes extensive use of PyStan which is licensed under the GPL license, a requirement of which is that derivative works are also licensed under the GPL license. Hence this notebook is distributed under the GPL license. There is a copy of the full text of the license in this directory.

<hr>

<small style="font-size:12px">
This notebook is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This notebook is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with Foobar.  If not, see <https://www.gnu.org/licenses/>.
</small>